# Manutenzione preventiva - 

# Manutenzione preventiva - DoS Attacks

In [42]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import functions_notebook as fn
import costants as cs
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
data_cc_30m = fn.open_convert_capture(".\\captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json")
data_cc_1h = fn.open_convert_capture(".\\captures\\captures1_v2\\clean\\eth2dump-clean-1h_1.json")

data_pf_att_1m_30m_1 = fn.open_convert_capture(".\\captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_2 = fn.open_convert_capture(".\\captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")
data_pf_att_1m_30m_3 = fn.open_convert_capture(".\\captures\\captures1_v2\\pingFloodDDoS\\eth2dump-pingFloodDDoS1m-0,5h_1.json")

In [24]:
pv_cc_30m = fn.create_array_values(data_cc_30m)
pv_cc_1h = fn.create_array_values(data_cc_1h)

pv_pf_att_1m_30m_1 = fn.create_array_values(data_pf_att_1m_30m_1)
pv_pf_att_1m_30m_2 = fn.create_array_values(data_pf_att_1m_30m_2)
pv_pf_att_1m_30m_3 = fn.create_array_values(data_pf_att_1m_30m_3)

In [48]:
print(len(cs.df_columns))

57


In [14]:
# Costruzione dataframe
# Conversione colonna timestamp a float
# Inserimento colonna inter packet arrival time
# Cattura clean 30m
df_cc_30m = pd.DataFrame(data=pv_cc_30m, columns=cs.df_columns)
df_cc_30m['frame.time_epoch'] = df_cc_30m['frame.time_epoch'].astype('float')
df_cc_30m[cs.ipat_field_name] = df_cc_30m['frame.time_epoch'].diff()

# Cattura clean 1h
df_cc_1h = pd.DataFrame(data=pv_cc_1h, columns=cs.df_columns)
df_cc_1h['frame.time_epoch'] = df_cc_1h['frame.time_epoch'].astype('float')
df_cc_1h[cs.ipat_field_name] = df_cc_1h['frame.time_epoch'].diff()

# Catture pf att 1m 30m
df1_pfatt_1m_30m_1 = pd.DataFrame(pv_pf_att_1m_30m_1, columns=cs.df_columns)
df2_pfatt_1m_30m_2 = pd.DataFrame(pv_pf_att_1m_30m_2, columns=cs.df_columns)
df3_pfatt_1m_30m_3 = pd.DataFrame(pv_pf_att_1m_30m_3, columns=cs.df_columns)

# Concatenazione per righe
df_pfatt_1m_30m = pd.concat([df1_pfatt_1m_30m_1, df2_pfatt_1m_30m_2, df3_pfatt_1m_30m_3], ignore_index=True)

ValueError: 63 columns passed, passed data had 69 columns

In [37]:
df_cc_30m = fn.clean_df(df_cc_30m)
df_cc_1h = fn.clean_df(df_cc_1h)
df_pfatt_1m_30m = fn.clean_df(df_pfatt_1m_30m)

In [38]:
print(df_pfatt_1m_30m.head())

       frame.time_epoch  eth.type  eth.len arp.hw.type arp.proto.type  \
0  1526898582.556747000    2048.0      NaN         NaN            NaN   
1  1526898582.566616000    2048.0      NaN         NaN            NaN   
2  1526898582.567459000    2048.0      NaN         NaN            NaN   
3  1526898582.567478000    2048.0      NaN         NaN            NaN   
4  1526898582.567656000    2048.0      NaN         NaN            NaN   

  arp.hw.size arp.proto.size icmp.type icmp.code icmp.checksum  ... udp.port  \
0         NaN            NaN       NaN       NaN           NaN  ...      NaN   
1         NaN            NaN       NaN       NaN           NaN  ...      NaN   
2         NaN            NaN       NaN       NaN           NaN  ...      NaN   
3         NaN            NaN       NaN       NaN           NaN  ...      NaN   
4         NaN            NaN       NaN       NaN           NaN  ...      NaN   

   udp.length  udp.checksum  udp.checksum_status  mbtcp.trans_id  \
0         Na